# 1 process vs 2, in computing

In [17]:
from time import time
from multiprocessing import Process, Value
from functools import wraps

NUMBER_OF_ITERS = 1_000_000  # Reduced for testing

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        elapsed = end_time - start_time
        print(f"Function '{func.__name__}' took {elapsed:.4f} seconds to execute.")
        return result
    return wrapper

def count(counter, increments):
    for _ in range(increments):
        with counter.get_lock():
            counter.value += 1

@measure_time
def one_process_count():
    counter = Value('i', 0)
    for _ in range(NUMBER_OF_ITERS):
        with counter.get_lock():
            counter.value += 1
    print(f"Single process finished. Final counter value: {counter.value}")

@measure_time
def two_processes_split_count():
    counter = Value('i', 0)
    half_increments = NUMBER_OF_ITERS // 2

    p1 = Process(target=count, args=(counter, half_increments))
    p2 = Process(target=count, args=(counter, half_increments))
    
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(f"Two processes finished. Final counter value: {counter.value}")

if __name__ == '__main__':
    print(f"Testing with {NUMBER_OF_ITERS} iterations")
    one_process_count()
    two_processes_split_count()

Testing with 1000000 iterations
Single process finished. Final counter value: 1000000
Function 'one_process_count' took 1.2517 seconds to execute.
Two processes finished. Final counter value: 0
Function 'two_processes_split_count' took 0.0921 seconds to execute.


In [23]:
import asyncio

async def say_hello(name):
    print(f"Hello {name}")
    await asyncio.sleep(1)
    print(f"Goodbye {name}")

async def main():
    tasks = [say_hello(name) for name in ["Alice", "Bob", "Charlie"]]
    await asyncio.gather(*tasks)

await(main())

Hello Alice
Hello Bob
Hello Charlie
Goodbye Alice
Goodbye Bob
Goodbye Charlie
